In [2]:
import torch
from torch import nn
import torchtext
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer
from torch.utils.data import DataLoader, BatchSampler, RandomSampler
import pandas as pd
import numpy as np
from tqdm import tqdm


In [3]:
# Custom DataPiper

train_data = pd.read_csv("data/cnn_dailymail/train.csv")
train_data

Need to design an encoder-decoder architecture and summarize text by mapping article and highlights

In [11]:
def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)
tokenizer = get_tokenizer('spacy')
embed = torchtext.vocab.GloVe("6B", 100)

/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(


In [12]:
vocab = torchtext.vocab.build_vocab_from_iterator(yield_tokens(train_data['article'].values), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [ ]:
article_pipeline = lambda x: vocab(tokenizer(x))
highlight_pipeline = lambda x: vocab(tokenizer(x))

In [ ]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
     highlight_list, article_list = [], []
     for (_highlight, _article) in batch:
          highlight_list.append(highlight_pipeline(_highlight))
          p_text = torch.tensor(article_pipeline(_article.lower()), dtype=torch.int64)
          article_list.append(p_text)
     # Pad text documents with lower than the max seq. length of the batch
     article_list = torch.nn.utils.rnn.pad_sequence(article_list, batch_first=True)
     highlight_list = torch.tensor(highlight_list, dtype=torch.float32).reshape(-1, 1)
     return highlight_list.to(device), article_list.to(device)


In [ ]:
class ThisIsCNN(torch.utils.data.Dataset):
    def __init__(self, data):
        self._articles = data['article'].values
        self._highlights = data['highlight'].values
    
    def __len__(self):
        return self.articles.shape[0]
    
    def __getitem__(self, idx):

        _article = self._articles[idx]
        _highlight = self._highlights[idx]

        return _highlight, _article

In [ ]:
class Encoder(nn.Module):
    def __init__(self, embedding_vector) -> None:
        super(Encoder, self).__init__()

        self.embed = torch.nn.Embedding.from_pretrained(embedding_vector, freeze=True)

        self.rnn = nn.LSTM(input_size=100, hidden_size=100, num_layers=2, batch_first=True, bidirectional=True)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(200, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.embed(x)
        
        _, (hidden,_) = self.rnn(x)

        # Picking only the values of the forward and backward layer(bidirectional) of hidden output
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = -1)
        logits = self.fc1(hidden)

        probs = self.sigmoid(logits)

        return logits, probs